# Detect Arbitrage
[link](https://www.algoexpert.io/questions/Detect%20Arbitrage)

## My Solution

In [ ]:
import math

# O(n^3) time | O(n^2) space
def detectArbitrage(exchangeRates):
    # Write your code here.
    exchange = [[-math.log(exchangeRates[i][j]) for j in range(len(exchangeRates))] for i in range(len(exchangeRates))]
    n = len(exchange)
    
    opt = [float('inf') for _ in exchange]
    opt[-1] = 0
    cache = [_ for _ in opt]
    
    for _ in range(n):
        for i in range(n):
            cache[i] = min([opt[j] + exchange[i][j] for j in range(n)])
            
        isSame = True
        for i in range(n):
            if cache[i] != opt[i]:
                isSame = False
                opt[i] = cache[i]
        if isSame:
            return False
    return True

In [ ]:
import math

def detectArbitrage(exchangeRates):
    # Write your code here.
	exchange = [[-math.log(exchangeRates[i][j]) for j in range(len(exchangeRates))] for i in range(len(exchangeRates))]
	n = len(exchange)
	
	opt = [float('inf') for _ in exchange]
	opt[-1] = 0
	cache = [_ for _ in opt]
	
	for _ in range(n):
		isSame = True
		for i in range(n):
			newMinDistance = min([opt[j] + exchange[i][j] for j in range(n)])
			if newMinDistance < opt[i]:
				opt[i] = newMinDistance
				isSame = False

		if isSame:
			return False
		
    return True

## Expert Solution

In [ ]:
import math

# O(n^3) time | O(n^2) space
def detectArbitrage(exchangeRates):
    # To use exchange rates as edge weights, we must be able to add them.
    # Since log(a*b) = log(a) + log(b), we can covert all rates to
    # -log10(rate) to use them as edge weights.
    logExchangeRates = convertToLogMatrix(exchangeRates)

    # A negative weight cycle indicates an arbitrage.
    return foundNegativeWeightCycle(logExchangeRates, 0)

# Runs the Bellman-Ford Algorithm to detect any negative-weight cycles.
def foundNegativeWeightCycle(graph, start):
    distancesFromStart = [float("inf") for _ in range(len(graph))]
    distancesFromStart[start] = 0

    for _ in range(len(graph) - 1):
        # If no update occurs, that means there's no negative cycle.
        if not relaxEdgesAndUpdateDistances(graph, distancesFromStart):
            return False

    return relaxEdgesAndUpdateDistances(graph, distancesFromStart)

# Returns `True` if any distance was updated
def relaxEdgesAndUpdateDistances(graph, distances):
    updated = False
    for sourceIdx, edges in enumerate(graph):
        for destinationIdx, edgeWeight in enumerate(edges):
            newDistanceToDestination = distances[sourceIdx] + edgeWeight
            if newDistanceToDestination < distances[destinationIdx]:
                updated = True
                distances[destinationIdx] = newDistanceToDestination
    
    return updated

def convertToLogMatrix(matrix):
    newMatrix = []
    for row, rates in enumerate(matrix):
        newMatrix.append([])
        for rate in rates:
            newMatrix[row].append(-math.log10(rate))

    return newMatrix

## Thoughts
### recurrence formula of bellman-ford algorithm
opt(i, v) - the shortest distance from node v to the destination node t using at most i steps
```
for (n - 1) iterations:
    for each node in the graph:
        opt(i, v) = min(
            opt(i - 1, v),
            min(opt(i - 1, w) + e(w -> v), for all node w's which are the "from" neighbors of node v )
        )
```

### why (n-1) iterations
- in a graph with n nodes, one node at most needs (n-1) steps to reach another node.
- i iterations can guarantee the nodes that needs at least i steps to the destination node get the correct answer in the opt table.


### the number of iterations
- by bellman-ford, finding the shortest path to a node in the graph needs (n - 1) iterations, where n is the number of nodes.
- how many more iterations that bellman-ford algorithm needs to detect the negative cycle in the graph?
- if the destination node can be reached by every other node through the edges, we just need to run one more iteration.
- because, if the destination node can be reached by every other node (the normal situation), (n - 1) iterations guarantee that the opt table converges, which means any value in opt table shouldn't change if we run one or more iterations. if some values in opt table change, this means there are negative cycles in the graph.
- but, if the destination node can't be reached from some nodes in the graph, the distance from those nodes to the destination node will always be infinite. in that situation, we can't detect if there's any negtive cycle in among those nodes. in this case, we can create a dummy node that every other node (including the destination node) points to, seeing it as the destination node, and the question will be reduced to the noremal situation.
- in the "detect arbitrage" question, any currency can exchange to any other currency, so the graph we create is fully connected, so we just need to run one more iteration to detect the negative cycle.

### early stop
- the bellman-ford algorithm can be early stopped if the opt table doesn't change between two consecutive iterations, because the opt table converges, more iterations won't change the values in the table.
- in normal situation, early stop means there's no negative cycles in the graph, because if there's any negative cycle, the distance between some nodes to the destination node will continuelly reduce.
- in our situation, the graph is fully connected, so if the opt table doesn't change between two consecutive iterations, we can conclude that there's no negtive cycle in the graph, which means there's no arbitrage in the given exchange table. otherwise, if the opt table still change at the #n iteration, there's must be a negative cycle in the graph, which leads to an arbitrage.

### change to the recurrence formula
in recurrence formula
```
opt(i, v) = min(
    opt(i - 1, v),
    min(opt(i - 1, w) + e(w -> v), for all node w's which are the "from" neighbors of node v )
)
```
we can seem `opt(i - 1, v)` as `opt(i - 1, w) + e(w -> v) where w is v`, `e(v -> v) == 0`, so the recurrence formula can be changed to:
```
opt(i, v) = min(opt(i - 1, w) + e(w -> v), for all node w's which are the "from" neighbors of node v and v itself)
```
because our graph is fully connected, any node's "from" neighbors and itself are all the nodes in the graph, so the formula can be modified more as:
```
opt(i, v) = min(opt(i - 1, w) + e(w -> v), for all nodes w in the graph)
```

### time complexity
- bellman-ford algorithm needs O(n*m) time - where n is the number of nodes and m is the number of edges. because our graph is fully connected, m = n^2, so the time complexity is O(n^3).
- bellman-ford algorithm needs O(n) space, but get the negtive logarithm adjacent table needs O(n^2) space, so totally O(n^2) space.

### other tips
- we can directly modify opt table instead of using a cache table, because every time modifying the opt table is at the moment we truely find a lower distance path from a node to the destination node. the later utilization of the modified opt table will be valid.